## Notebook Imports 

In [796]:
from numpy.random import seed
seed(888)
from tensorflow import random
random.set_seed(404)

In [797]:
import os
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from time import strftime
tf.reset_default_graph()
from PIL import Image 

## Constants 

In [798]:
X_TRAIN_PATH = '/home/goodluck/Desktop/Data/Neural Networks/HandwritingRecognition_project/MNIST/digit_xtrain.csv'
X_TEST_PATH = '/home/goodluck/Desktop/Data/Neural Networks/HandwritingRecognition_project/MNIST/digit_xtest.csv'
Y_TRAIN_PATH = '/home/goodluck/Desktop/Data/Neural Networks/HandwritingRecognition_project/MNIST/digit_ytrain.csv'
Y_TEST_PATH = '/home/goodluck/Desktop/Data/Neural Networks/HandwritingRecognition_project/MNIST/digit_ytest.csv'
NR_CLASSES = 10
VALIDATION_SIZE = 10000

LOGGING_PATH = '/home/goodluck/Desktop/Data/Neural Networks/HandwritingRecognition_project/tensorboard_mnsit_digits_logs/'

IMAGE_HEIGHT = 28
IMAGE_WIDTH = 28
NR_CHANNELS = 1
TOTAL_INPUTS = IMAGE_HEIGHT*IMAGE_WIDTH*NR_CHANNELS

## Load the Data

In [799]:
%%time
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype = int)

CPU times: user 221 ms, sys: 127 µs, total: 221 ms
Wall time: 221 ms


In [800]:
%%time
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype = int)

CPU times: user 42.4 ms, sys: 0 ns, total: 42.4 ms
Wall time: 42.9 ms


In [801]:
%%time
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype = int)

CPU times: user 43.3 s, sys: 1.04 s, total: 44.3 s
Wall time: 14min 7s


In [802]:
%%time
x_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype = int)

CPU times: user 39.7 ms, sys: 370 µs, total: 40.1 ms
Wall time: 39.4 ms


In [803]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

## Explore 

In [804]:
y_train_all.shape

(60000,)

In [805]:
x_train_all.shape

(60000, 784)

In [806]:
y_test.shape

(10000,)

In [807]:
y_test.shape

(10000,)

In [808]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

### Data pre processing

In [809]:
x_train_all , x_test = x_train_all / 255.0 ,  x_test / 255.0 

In [810]:
values = y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [811]:
np.eye(10)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

### Convert into one hot encoding

In [812]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [813]:
y_train_all.shape

(60000, 10)

In [814]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

### Create a validation Dataset from training Data

In [815]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [816]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [817]:
x_train.shape

(50000, 784)

In [818]:
x_val.shape

(10000, 784)

## Set up Tensor flow Graph 

In [819]:
x = tf.placeholder(tf.float32, shape = [None, TOTAL_INPUTS])
y = tf.placeholder(tf.float32, shape = [None, NR_CLASSES])

In [820]:
x

<tf.Tensor 'Placeholder:0' shape=(?, 784) dtype=float32>

## Neural Network Architecture

### Hyper parameters

In [821]:
nr_epochos = 5 
learning_rate = 1e-4

n_hidden1 = 512
n_hidden2 = 64

In [822]:
initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42)
w1 = tf.Variable(initial_value= initial_w1)


In [823]:
initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
b1 = tf.Variable(initial_value = initial_b1)

In [824]:
layer1_in = tf.matmul(x, w1) + b1

In [825]:
layer1_out = tf.nn.relu(layer1_in)

In [826]:
initial_w2 = tf.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
w2 = tf.Variable(initial_value= initial_w2)

initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
b2 = tf.Variable(initial_value = initial_b2)

layer2_in = tf.matmul(layer1_out, w2) + b2
layer2_out = tf.nn.relu(layer2_in)


In [827]:
initial_w3 = tf.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42)
w3 = tf.Variable(initial_value= initial_w3)

initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
b3 = tf.Variable(initial_value = initial_b3)

layer3_in = tf.matmul(layer2_out, w3) + b3
output = tf.nn.softmax(layer3_in)


## Tensor Board Setup

In [828]:
# Folder for Tensor Board

folder_name = f'Model 1 at {strftime("%H:%M")}'
directory= os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else: 
    print("Sucessful created directory")

Sucessful created directory


## Loss, optimisation & metrics

### defining loss function

In [829]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output))

### Defining Optimizier

In [830]:
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(loss)

### Accuracy Metric

In [831]:
correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [832]:
tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('cost', loss)

<tf.Tensor 'cost:0' shape=() dtype=string>

# Run Session 

In [833]:
sess = tf.Session()


### set up filewriter and merge summaries

In [834]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

In [835]:
init = tf.global_variables_initializer()
sess.run(init)

In [836]:
w1.eval(sess)

array([[-0.02807751, -0.01377521, -0.06763297, ..., -0.02663724,
         0.02861341, -0.05550233],
       [-0.16205125, -0.18859725, -0.03102448, ..., -0.0820701 ,
        -0.03345905, -0.02453214],
       [ 0.12582639, -0.16444902,  0.13603579, ..., -0.09897225,
        -0.09923435,  0.1451435 ],
       ...,
       [-0.04471837, -0.09593774, -0.08978765, ...,  0.04240045,
        -0.18997248,  0.00134785],
       [ 0.03215451,  0.04336654, -0.18240118, ...,  0.08296242,
        -0.10039439, -0.12682591],
       [ 0.08766606, -0.15083945,  0.08048793, ...,  0.07548849,
        -0.04359084, -0.11031353]], dtype=float32)

## batching the Data

In [837]:
size_of_batch = 1000

In [838]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0 

In [839]:
def next_batch (batch_size, data, labels):
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    end= index_in_epoch
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    return data[start:end], labels[start:end]

## Training Loop 

In [840]:

for epoch in range(nr_epochos):
    for i in range(nr_iterations):
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dictionary = {x:batch_x, y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
    
    train_writer.add_summary(s, epoch)
        
    print(f'Epoch {epoch} \t | Training Accuracy = {batch_accuracy}')
    
    
    summary = sess.run(fetches=merged_summary, feed_dict={x:x_val, y:y_val})
    validation_writer.add_summary(summary, epoch)
    
print("Done training")
        
        

Epoch 0 	 | Training Accuracy = 0.4050000011920929
Epoch 1 	 | Training Accuracy = 0.7329999804496765
Epoch 2 	 | Training Accuracy = 0.8209999799728394
Epoch 3 	 | Training Accuracy = 0.843999981880188
Epoch 4 	 | Training Accuracy = 0.8560000061988831
Done training


## Make a Prediction 

In [841]:
img = Image.open('MNIST/test_img.png')
img

In [842]:
bw = img.convert('L')

In [843]:
img_array = np.invert(bw)

In [844]:
img_array.shape

(28, 28)

In [845]:
test_img = img_array.ravel()

In [846]:
test_img.shape

(784,)

In [847]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={x: [test_img]})

In [849]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={x:[test_img]})

## Testing and Evaluating

In [850]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={x:x_test, y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

ValueError: Cannot feed value of shape (10000,) for Tensor 'Placeholder:0', which has shape '(?, 784)'

In [ ]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()